In [5]:
import json
import pandas as pd
import requests
import yfinance as yf
from datetime import date, timedelta
import numpy as np

In [93]:
stock = "MSFT"
infer_request = requests.get(f"http://127.0.0.1:8003/inference/{stock}")
infer_string = json.loads(infer_request.text)
new_datapoint = []


In [94]:
td = date.today()
infer_dict = {}
infer_array = []
i = 0
for key, val in infer_string.items():
    infer_dict['Date'] = td
    infer_dict['Predicted'] = val
    infer_array.append(infer_dict)
    infer_dict = {}
    if i != 0:
        new_datapoint.append(td)
    # print(td, val)
    td = td+timedelta(days=1)
    while td.weekday() in [5, 6]:
        td = td+timedelta(days=1)
    i += 1
# infer_array
infer_dataframe = pd.DataFrame(infer_array)
infer_dataframe


,Date,Predicted
0,2023-04-29,271.905630
1,2023-05-01,272.366095
2,2023-05-02,272.704823
3,2023-05-03,272.892051
4,2023-05-04,272.920664


In [95]:
new_datapoint

[datetime.date(2023, 5, 1),
 datetime.date(2023, 5, 2),
 datetime.date(2023, 5, 3),
 datetime.date(2023, 5, 4)]

In [100]:
stock = "MSFT"
request = requests.get(f"http://127.0.0.1:8003/lstm_data_view/{stock}")
js_string = json.loads(request.text)


In [101]:
# Use pandas.DataFrame.from_dict() to Convert JSON to DataFrame
predict_df = pd.read_json(request.text, orient='index')
predict_df.columns = ["Actual", "Date", "Predicted"]
predict_df.Date = predict_df.Date.dt.date
predict_df.tail(5)


,Actual,Date,Predicted
844,281.77,2023-04-24,269.008
845,275.42,2023-04-25,269.647
846,295.37,2023-04-26,270.193
847,304.83,2023-04-27,270.609
848,307.26,2023-04-28,270.980


In [102]:
predict_df = predict_df.tail(5)
predict_df

,Actual,Date,Predicted
844,281.77,2023-04-24,269.008
845,275.42,2023-04-25,269.647
846,295.37,2023-04-26,270.193
847,304.83,2023-04-27,270.609
848,307.26,2023-04-28,270.980


In [103]:
predict_df = pd.concat([predict_df, infer_dataframe], ignore_index=True).rename(columns={"Actual":"actual", "Date":"date", "Predicted":"predicted"})

In [104]:
predict_df

,actual,date,predicted
0,281.77,2023-04-24,269.008000
1,275.42,2023-04-25,269.647000
2,295.37,2023-04-26,270.193000
3,304.83,2023-04-27,270.609000
4,307.26,2023-04-28,270.980000
5,NaN,2023-04-29,271.905630
6,NaN,2023-05-01,272.366095
7,NaN,2023-05-02,272.704823
8,NaN,2023-05-03,272.892051
9,NaN,2023-05-04,272.920664


In [78]:
predict_df["date"] = pd.to_datetime(predict_df.date).dt.strftime('%d %b')

In [81]:
predict_df.date.dt.strftime('%d %b')

0    24 Apr
1    25 Apr
2    26 Apr
3    27 Apr
4    28 Apr
5    29 Apr
6    01 May
7    02 May
8    03 May
9    04 May
Name: date, dtype: object

In [85]:
predict_df["actual_change"] = predict_df.actual.diff(1)
predict_df["predicted_change"] = predict_df.predicted.diff(1)
predict_df["actual_%change"] = predict_df.actual.pct_change(fill_method="bfill")
predict_df["predicted_%change"] = predict_df.predicted.pct_change(fill_method="bfill")
predict_df["date"] = pd.to_datetime(predict_df.date).dt.strftime('%d %b')
predict_df = predict_df.fillna("")
predict_df

,actual,date,predicted,actual_change,predicted_change,actual_%change,predicted_%change
0,281.77,24 Apr,269.008000,,,,
1,275.42,25 Apr,269.647000,-6.35,0.639,-0.022536,0.002375
2,295.37,26 Apr,270.193000,19.95,0.546,0.072435,0.002025
3,304.83,27 Apr,270.609000,9.46,0.416,0.032028,0.00154
4,307.26,28 Apr,270.980000,2.43,0.371,0.007972,0.001371
5,,29 Apr,271.905630,,0.92563,,0.003416
6,,01 May,272.366095,,0.460464,,0.001693
7,,02 May,272.704823,,0.338728,,0.001244
8,,03 May,272.892051,,0.187228,,0.000687
9,,04 May,272.920664,,0.028613,,0.000105


In [19]:
st = yf.Ticker(stock)
periods = {"1d": "1D",
           "5d": "5D",
           "1mo": "1M",
           "6mo": "6M",
           "ytd": "YTD",
           "1y": "1Y",
           "5y": "5Y",
           "max": "Max"}
intervals = {"1d": "1m",
             "5d": "15m",  # 15m
             "1mo": "90m",
             "6mo": "1d",
             "ytd": "1d",
             "1y": "1d",
             "5y": "1wk",
             "max": "3mo"}
period = "6mo"
interval = intervals[period]
# print(interval)
# Create a line graph
df = st.history(period=(period), interval=interval)
df = df.reset_index()

df.columns = ['Date-Time']+list(df.columns[1:])
df["Date"] = df["Date-Time"].dt.date


In [20]:


df = pd.concat([df, pd.DataFrame(new_datapoint, columns=['Date'])])


In [21]:
df = df.reset_index().drop(columns=["index"])

In [23]:
df_date = pd.to_datetime(df['Date'])
df_date = df_date.dt.tz_localize('US/Eastern').dt.tz_convert('US/Eastern')
df_date


0     2022-10-24 00:00:00-04:00
1     2022-10-25 00:00:00-04:00
2     2022-10-26 00:00:00-04:00
3     2022-10-27 00:00:00-04:00
4     2022-10-28 00:00:00-04:00
                 ...           
124   2023-04-24 00:00:00-04:00
125   2023-04-25 00:00:00-04:00
126   2023-04-26 00:00:00-04:00
127   2023-04-27 00:00:00-04:00
128   2023-04-28 00:00:00-04:00
Name: Date, Length: 129, dtype: datetime64[ns, US/Eastern]

In [24]:
df['Date-Time'].fillna(df_date, inplace = True)


In [25]:
df.tail(6)

,Date-Time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
123,2023-04-21 00:00:00-04:00,285.010010,286.269989,283.059998,285.760010,21665400.0,0.0,0.0,2023-04-21
124,2023-04-24 00:00:00-04:00,282.089996,284.940002,278.720001,281.730011,23781865.0,0.0,0.0,2023-04-24
125,2023-04-25 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-25
126,2023-04-26 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-26
127,2023-04-27 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-27
128,2023-04-28 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-28


In [26]:
df = pd.merge(df, predict_df, how="left", on="Date")


In [27]:
df.tail(6)

,Date-Time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Actual,Predicted
123,2023-04-21 00:00:00-04:00,285.010010,286.269989,283.059998,285.760010,21665400.0,0.0,0.0,2023-04-21,285.76,265.076000
124,2023-04-24 00:00:00-04:00,282.089996,284.940002,278.720001,281.730011,23781865.0,0.0,0.0,2023-04-24,NaN,266.412861
125,2023-04-25 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-25,NaN,266.888085
126,2023-04-26 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-26,NaN,267.182780
127,2023-04-27 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-27,NaN,267.289946
128,2023-04-28 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-28,NaN,267.215979


In [ ]:
# df[~df['Predicted'].isnull().any(axis=1)]
df[df['Predicted']>0]
# df[~df.isnull().any(axis=1)]
